## STEP 2

in this step we build a network bellonging to Vattenfall 'Island - Study Case.docx'. If we look at the one line diagram provided by them this network consist of:

- 5 trafos

- 18 bus bars

- 7 loads

- 12 cables

This quantities are still manageable for copy parting all of the commands you learned in the previous steps but we will try to do this in a more automatic way using For loops


 ## making this process automatic relies in having DATA to read. I have already done this step in the exCel file that you find in canvas within this step

## download the file called my_network and check the information in all the tabs:
- my_lines
- my_trafos
- my_buses
- my_loads
- my_cables

do they make sense to you with regards to the island one line diagram?

## save the file in the same folder where you have THIS jupyter notebook (or the notebook where you are running the commands)

# LOAD AND READ THE FILES HERE
Most probably you will have to select the Python for the Kernel again

In [1]:
import pandas as pd
import openpyxl

load_data=pd.read_excel('my_network.xlsx',sheet_name='my_loads') 
bus_data=pd.read_excel('my_network.xlsx',sheet_name='my_buses') 
trafo_data=pd.read_excel('my_network.xlsx',sheet_name='my_trafos') 
line_data=pd.read_excel('my_network.xlsx',sheet_name='my_lines') 

## Create a For loop (need it to do the network building automatic)
Creating a for loop in python follows the same logic as in any other programming language. The only difference is the syntax.
Have a look at the for loops below, I suggest you copy line by line and use the function "PRINT" progressively to know what is going on "inside" the For loop. Then make it more complex (adding more lines of code) as you go!

In [2]:
# example of a simple for loop
for m in range(0,10): #for the counter m which is in the range from 0 to 10...do a for loop
    print(m)   # here I simply put the m counter to print so you see it is actually happening

0
1
2
3
4
5
6
7
8
9


In [3]:
# example of a for loop with dataframes (the data from above)
for ind in trafo_data.index: # for the counter ind which is iterating over the indexes of the dataframe trafo_data
    print(ind)

0
1
2
3
4


In [4]:
for ind2 in trafo_data.columns: # for the counter ind2 that is iterating over the columns of the trafo_data datafrape
    print(ind2)

From
To
Name
Type


In [5]:
for lidx, b in bus_data.iterrows(): 
    #this one is a bit more complew, the counter "b" has all info of one row of bus_data
    #see what happens if you change to print(b.Name) below
    print(b)

Name          SS1_0
Voltage_kV     11.0
Name: 0, dtype: object
Name          SS1_1
Voltage_kV      0.4
Name: 1, dtype: object
Name          SS1_2
Voltage_kV      0.4
Name: 2, dtype: object
Name          SS2_0
Voltage_kV     11.0
Name: 3, dtype: object
Name          SS2_1
Voltage_kV      0.4
Name: 4, dtype: object
Name          SS2_2
Voltage_kV      0.4
Name: 5, dtype: object
Name          SS3_0
Voltage_kV     11.0
Name: 6, dtype: object
Name          SS3_1
Voltage_kV      0.4
Name: 7, dtype: object
Name          SS3_2
Voltage_kV      0.4
Name: 8, dtype: object
Name          SS4_0
Voltage_kV     11.0
Name: 9, dtype: object
Name          SS4_1
Voltage_kV      0.4
Name: 10, dtype: object
Name          SS4_2
Voltage_kV      0.4
Name: 11, dtype: object
Name          SS4_3
Voltage_kV      0.4
Name: 12, dtype: object
Name          SS5_0
Voltage_kV     11.0
Name: 13, dtype: object
Name          SS5_1
Voltage_kV      0.4
Name: 14, dtype: object
Name          SS5_2
Voltage_kV      0.4
Name: 15, 

You get the point :) if not...ask...below are more examples of for loops

Remember: USE PRINT!, it helps to see what is going on!

## Create actual network elements

In [7]:
import pandapower as pp
# empty network
net=pp.create_empty_network()

# buses
for _, b in bus_data.iterrows(): 
    pp.create_bus(net, name=b.Name, vn_kv=b.Voltage_kV)
    
# lines
for lidx, linerow in line_data.iterrows():
    TB=line_data.To[lidx] 
    FB=line_data.From[lidx]
    lin_name=line_data.Name[lidx]
    lin_type=line_data.Type[lidx]
    
    A=net.bus.index[net.bus['name']==FB][0] #this is one way to access info in a dataframe
    B=pp.get_element_index(net,"bus",TB) #this is a in-build funciton in pandapower that does the same
    C=line_data.Length_kms[lidx]
    
    pp.create_line(net, from_bus=A, to_bus=B, length_km=C, std_type=lin_type, name=lin_name)
    
    
# trafos
for lidx, linerow in trafo_data.iterrows():
    TB=trafo_data.To[lidx]
    FB=trafo_data.From[lidx]
    A=pp.get_element_index(net,"bus",FB)
    B=pp.get_element_index(net,"bus",TB)
    pp.create_transformer(net,A,B,name=trafo_data.Name[lidx],std_type=trafo_data.Type[lidx])

    
#external grid
EG=pp.get_element_index(net,"bus","EG")
extgrid=pp.create_ext_grid(net,EG, voltage = 1.0, va_degree=0,name="External Grid")

see what I have built unit now:

In [8]:
net

This pandapower network includes the following parameter tables:
   - bus (18 elements)
   - ext_grid (1 element)
   - line (12 elements)
   - trafo (5 elements)

only missing the loads, I will add them manually because I am lazy, but you can try to add them similar to the for loops above (e.g. use the "load_data" information)

In [9]:
# loads
load1=pp.create_load(net, bus=2, p_mw=0.10, q_mvar=0.01, name="Load 1")
load2=pp.create_load(net, bus=5, p_mw=0.10, q_mvar=0.01, name="Load 2")
load3=pp.create_load(net, bus=8, p_mw=0.10, q_mvar=0.01, name="Load 3")
load4=pp.create_load(net, bus=11, p_mw=0.10, q_mvar=0.01, name="Load 4")
load5=pp.create_load(net, bus=12, p_mw=0.05, q_mvar=0.01, name="Load 5")
load6=pp.create_load(net, bus=15, p_mw=0.10, q_mvar=0.01, name="Load 6")
load7=pp.create_load(net, bus=16, p_mw=0.04, q_mvar=0.01, name="Load 7")

In [10]:
net

This pandapower network includes the following parameter tables:
   - bus (18 elements)
   - load (7 elements)
   - ext_grid (1 element)
   - line (12 elements)
   - trafo (5 elements)

# we now run a power flow of the network

In [11]:
pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [12]:
net.res_trafo.loading_percent #the results of the trafo tells us how much the trasformer is loaded

0    37.637761
1    37.557112
2    37.574006
3    57.224455
4    53.344826
Name: loading_percent, dtype: float64

In [13]:
net.res_line.loading_percent #same for the lines

0     22.596635
1     18.785473
2     14.999044
3     11.217329
4      5.422297
5     94.982476
6     94.778417
7     94.822787
8     96.544597
9     48.369390
10    96.183294
11    39.011609
Name: loading_percent, dtype: float64

# the load of some lines seems to be very high (>90%)

we can reinforce the grid them by adding more cables

In [14]:
#this parameter shows how many parallel cables you have in one line
net.line.parallel

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
Name: parallel, dtype: uint32

In [15]:
#I change this number to 2 (reinforce the network)
net.line.parallel.loc[5]=2
net.line.parallel.loc[6]=2
net.line.parallel.loc[7]=2
net.line.parallel.loc[8]=2
net.line.parallel.loc[10]=2

In [16]:
#here you see the change
net.line.parallel

0     1
1     1
2     1
3     1
4     1
5     2
6     2
7     2
8     2
9     1
10    2
11    1
Name: parallel, dtype: uint32

# run another power flow

In [17]:
pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [18]:
# check the line percentage
net.res_line.loading_percent

0     22.488818
1     18.697879
2     14.924862
3     11.153293
4      5.393988
5     47.237908
6     47.220591
7     47.283772
8     47.828571
9     48.363720
10    47.739956
11    39.007447
Name: loading_percent, dtype: float64

# HOW TO USE FOR LOOPS FOR RUNNING AN ENTIRE DAY?

in the project you will be receiving your own network so you don't necesarily need to spend a lot of time automatizing how to build elements (since it only happens once and then is just possibily some reinforcing)

however, it is important that you can run a TIMESERIES (the load changing hour per hour)

pandapower has a very good functionality for running time series you can follow it in this tutorial:

https://github.com/e2nIEE/pandapower/blob/develop/tutorials/time_series.ipynb 

if you have extra time one this tutorial we can do a quick loop over some random loads for this network, just let me know when you are in that step!